In [152]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras import Sequential
from sklearn.metrics import accuracy_score


In [13]:
data = pd.read_csv('train (2).csv')
data

,id,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,0,Aaradhya,Female,49.0,Ludhiana,Working Professional,Chef,NaN,5.0,NaN,NaN,2.0,More than 8 hours,Healthy,BHM,No,1.0,2.0,No,0
1,1,Vivan,Male,26.0,Varanasi,Working Professional,Teacher,NaN,4.0,NaN,NaN,3.0,Less than 5 hours,Unhealthy,LLB,Yes,7.0,3.0,No,1
2,2,Yuvraj,Male,33.0,Visakhapatnam,Student,NaN,5.0,NaN,8.97,2.0,NaN,5-6 hours,Healthy,B.Pharm,Yes,3.0,1.0,No,1
3,3,Yuvraj,Male,22.0,Mumbai,Working Professional,Teacher,NaN,5.0,NaN,NaN,1.0,Less than 5 hours,Moderate,BBA,Yes,10.0,1.0,Yes,1
4,4,Rhea,Female,30.0,Kanpur,Working Professional,Business Analyst,NaN,1.0,NaN,NaN,1.0,5-6 hours,Unhealthy,BBA,Yes,9.0,4.0,Yes,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140695,140695,Vidya,Female,18.0,Ahmedabad,Working Professional,NaN,NaN,5.0,NaN,NaN,4.0,5-6 hours,Unhealthy,Class 12,No,2.0,4.0,Yes,1
140696,140696,Lata,Female,41.0,Hyderabad,Working Professional,Content Writer,NaN,5.0,NaN,NaN,4.0,7-8 hours,Moderate,B.Tech,Yes,6.0,5.0,Yes,0
140697,140697,Aanchal,Female,24.0,Kolkata,Working Professional,Marketing Manager,NaN,3.0,NaN,NaN,1.0,More than 8 hours,Moderate,B.Com,No,4.0,4.0,No,0
140698,140698,Prachi,Female,49.0,Srinagar,Working Professional,Plumber,NaN,5.0,NaN,NaN,2.0,5-6 hours,Moderate,ME,Yes,10.0,1.0,No,0


In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140700 entries, 0 to 140699
Data columns (total 20 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   id                                     140700 non-null  int64  
 1   Name                                   140700 non-null  object 
 2   Gender                                 140700 non-null  object 
 3   Age                                    140700 non-null  float64
 4   City                                   140700 non-null  object 
 5   Working Professional or Student        140700 non-null  object 
 6   Profession                             104070 non-null  object 
 7   Academic Pressure                      27897 non-null   float64
 8   Work Pressure                          112782 non-null  float64
 9   CGPA                                   27898 non-null   float64
 10  Study Satisfaction                     27897 non-null   

In [19]:
for col in ['Academic Pressure', 'Work Pressure', 'CGPA', 'Study Satisfaction', 'Job Satisfaction']:
    data[col] = data[col].fillna(data[col].median())
data['Profession'] = data['Profession'].fillna(data['Profession'].mode())

In [20]:
data = data.dropna()

In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 104068 entries, 0 to 140698
Data columns (total 20 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   id                                     104068 non-null  int64  
 1   Name                                   104068 non-null  object 
 2   Gender                                 104068 non-null  object 
 3   Age                                    104068 non-null  float64
 4   City                                   104068 non-null  object 
 5   Working Professional or Student        104068 non-null  object 
 6   Profession                             104068 non-null  object 
 7   Academic Pressure                      104068 non-null  float64
 8   Work Pressure                          104068 non-null  float64
 9   CGPA                                   104068 non-null  float64
 10  Study Satisfaction                     104068 non-null  float

In [142]:
y = data.Depression
X = data.drop(['Depression', 'id', 'Name', 'City', 'Profession'], axis =1)


In [143]:
ohe = OneHotEncoder(handle_unknown='ignore').fit(X[X.select_dtypes('object').columns])

In [144]:
scl = MinMaxScaler().fit(X.drop(X.select_dtypes('object').columns, axis =1))

In [117]:
def data_processing(X, ohe, scl):
    data_cat = X.select_dtypes('object').columns
    X_ohe = ohe.transform(X[data_cat]).toarray()
    X_real = scl.transform(X.drop(data_cat, axis = 1))
    res = np.hstack([X_real, X_ohe])
    return res

In [118]:
X = data_processing(X, ohe, scl)

In [119]:
X.shape

(104068, 145)

In [120]:
model = Sequential()
model.add(Dense(100, input_shape= (X.shape[1],), activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(100, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

C:\Program Files\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_16 (Dense)                     │ (None, 100)                 │          14,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 100)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 100)                 │          10,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_18 (Dense)                     │ (None, 1)                   │             101 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 24,801 (96.88 KB)

 Trainable params: 24,801 (96.88 KB)

 Non-trainable params: 0 (0.00 B)

In [121]:
model.fit(X, y, epochs = 5, validation_split = 0.2)

Epoch 1/5
2602/2602 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.9580 - loss: 0.1251 - val_accuracy: 0.9679 - val_loss: 0.0913
Epoch 2/5
2602/2602 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9670 - loss: 0.0935 - val_accuracy: 0.9675 - val_loss: 0.0905
Epoch 3/5
2602/2602 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9674 - loss: 0.0914 - val_accuracy: 0.9677 - val_loss: 0.0893
Epoch 4/5
2602/2602 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9679 - loss: 0.0912 - val_accuracy: 0.9678 - val_loss: 0.0893
Epoch 5/5
2602/2602 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9673 - loss: 0.0916 - val_accuracy: 0.9684 - val_loss: 0.0890


In [166]:
test = pd.read_csv('test (1).csv')

In [167]:
ids = test.id

In [168]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93800 entries, 0 to 93799
Data columns (total 19 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   id                                     93800 non-null  int64  
 1   Name                                   93800 non-null  object 
 2   Gender                                 93800 non-null  object 
 3   Age                                    93800 non-null  float64
 4   City                                   93800 non-null  object 
 5   Working Professional or Student        93800 non-null  object 
 6   Profession                             69168 non-null  object 
 7   Academic Pressure                      18767 non-null  float64
 8   Work Pressure                          75022 non-null  float64
 9   CGPA                                   18766 non-null  float64
 10  Study Satisfaction                     18767 non-null  float64
 11  Jo

In [169]:
for col in ['Academic Pressure', 'Work Pressure', 'CGPA', 'Study Satisfaction', 'Job Satisfaction']:
    test[col] = test[col].fillna(test[col].median())
test['Profession'] = test['Profession'].fillna(test['Profession'].mode()[0])
test['Degree'] = test['Degree'].fillna(test['Degree'].mode()[0])
test['Dietary Habits'] = test['Dietary Habits'].fillna(test['Dietary Habits'].mode()[0])

test['Have you ever had suicidal thoughts ?'] = test['Have you ever had suicidal thoughts ?'].fillna(test['Have you ever had suicidal thoughts ?'].mode()[0])


In [170]:
test = test.drop(['id', 'Name'], axis =1)


In [171]:
test = test.drop(['City', 'Profession'], axis =1)


In [128]:
X_test = data_processing(test, ohe, scl)

In [179]:
predictions = model.predict(test)

In [173]:
predictions

array([0, 0, 0, ..., 0, 1, 0], dtype=int64)

In [ ]:
# Получение классов (0 или 1)
class_predictions = (predictions > 0.5).astype(int)

In [180]:
res = pd.DataFrame(predictions, columns = ['Depression'])

In [181]:
res = pd.concat([ids, res], axis = 1)

In [185]:
res.to_csv('prediction.csv', index=False)


In [186]:
hmm = pd.read_csv('prediction.csv')
hmm

,id,Depression
0,140700,0
1,140701,0
2,140702,0
3,140703,1
4,140704,0
...,...,...
93795,234495,0
93796,234496,1
93797,234497,0
93798,234498,1


In [141]:
data

,id,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,0,Aaradhya,Female,49.0,Ludhiana,Working Professional,Chef,3.0,5.0,7.77,3.0,2.0,More than 8 hours,Healthy,BHM,No,1.0,2.0,No,0
1,1,Vivan,Male,26.0,Varanasi,Working Professional,Teacher,3.0,4.0,7.77,3.0,3.0,Less than 5 hours,Unhealthy,LLB,Yes,7.0,3.0,No,1
3,3,Yuvraj,Male,22.0,Mumbai,Working Professional,Teacher,3.0,5.0,7.77,3.0,1.0,Less than 5 hours,Moderate,BBA,Yes,10.0,1.0,Yes,1
4,4,Rhea,Female,30.0,Kanpur,Working Professional,Business Analyst,3.0,1.0,7.77,3.0,1.0,5-6 hours,Unhealthy,BBA,Yes,9.0,4.0,Yes,0
5,5,Vani,Female,59.0,Ahmedabad,Working Professional,Finanancial Analyst,3.0,2.0,7.77,3.0,5.0,5-6 hours,Healthy,MCA,No,7.0,5.0,No,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140693,140693,Shaurya,Male,55.0,Srinagar,Working Professional,Data Scientist,3.0,1.0,7.77,3.0,3.0,Less than 5 hours,Unhealthy,M.Tech,No,9.0,2.0,No,0
140694,140694,Ishaani,Female,45.0,Ahmedabad,Working Professional,Teacher,3.0,2.0,7.77,3.0,5.0,Less than 5 hours,Moderate,B.Ed,Yes,1.0,5.0,No,0
140696,140696,Lata,Female,41.0,Hyderabad,Working Professional,Content Writer,3.0,5.0,7.77,3.0,4.0,7-8 hours,Moderate,B.Tech,Yes,6.0,5.0,Yes,0
140697,140697,Aanchal,Female,24.0,Kolkata,Working Professional,Marketing Manager,3.0,3.0,7.77,3.0,1.0,More than 8 hours,Moderate,B.Com,No,4.0,4.0,No,0


In [145]:
X

,Gender,Age,Working Professional or Student,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness
0,Female,49.0,Working Professional,3.0,5.0,7.77,3.0,2.0,More than 8 hours,Healthy,BHM,No,1.0,2.0,No
1,Male,26.0,Working Professional,3.0,4.0,7.77,3.0,3.0,Less than 5 hours,Unhealthy,LLB,Yes,7.0,3.0,No
3,Male,22.0,Working Professional,3.0,5.0,7.77,3.0,1.0,Less than 5 hours,Moderate,BBA,Yes,10.0,1.0,Yes
4,Female,30.0,Working Professional,3.0,1.0,7.77,3.0,1.0,5-6 hours,Unhealthy,BBA,Yes,9.0,4.0,Yes
5,Female,59.0,Working Professional,3.0,2.0,7.77,3.0,5.0,5-6 hours,Healthy,MCA,No,7.0,5.0,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140693,Male,55.0,Working Professional,3.0,1.0,7.77,3.0,3.0,Less than 5 hours,Unhealthy,M.Tech,No,9.0,2.0,No
140694,Female,45.0,Working Professional,3.0,2.0,7.77,3.0,5.0,Less than 5 hours,Moderate,B.Ed,Yes,1.0,5.0,No
140696,Female,41.0,Working Professional,3.0,5.0,7.77,3.0,4.0,7-8 hours,Moderate,B.Tech,Yes,6.0,5.0,Yes
140697,Female,24.0,Working Professional,3.0,3.0,7.77,3.0,1.0,More than 8 hours,Moderate,B.Com,No,4.0,4.0,No


In [149]:
ds_x_train, ds_x_test, ds_y_train, ds_y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [178]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth=4,
    l2_leaf_reg=0.2,
    loss_function='Logloss',
    eval_metric='AUC',
    random_seed=42,
    cat_features=list(X.select_dtypes('object').columns),  # Индексы категориальных признаков
    verbose=100,
    early_stopping_rounds = 10
)

# Обучение модели
model.fit(ds_x_train, ds_y_train)


0:	total: 88ms	remaining: 1m 27s
100:	total: 9.09s	remaining: 1m 20s
200:	total: 17.9s	remaining: 1m 11s
300:	total: 26.8s	remaining: 1m 2s
400:	total: 35.6s	remaining: 53.2s
500:	total: 44.4s	remaining: 44.2s
600:	total: 53.3s	remaining: 35.4s
700:	total: 1m 2s	remaining: 26.5s
800:	total: 1m 10s	remaining: 17.6s
900:	total: 1m 19s	remaining: 8.73s
999:	total: 1m 28s	remaining: 0us


In [156]:
y_pred = model.predict(ds_x_test)

# Вычисление точности
accuracy = accuracy_score(ds_y_test, y_pred)

In [157]:
accuracy

0.9664168348227155